In [138]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
from src.nodes import *
from src.mcmc import *

def get_stats(district_type):
    nodes_tbl = f'{proj_id}.redistricting_data.nodes_TX_2020_cntyvtd_{district_type}_contract10'
#     nodes_tbl = f'{proj_id}.redistricting_data.nodes_TX_2020_cntyvtd_{district_type}'
    M = MCMC(random_seed=9000000, nodes_tbl=nodes_tbl)
    M.post_process1()
    M.post_process2()
    query = f"select * from {M.hash_tbl}"
    df = run_query(query)
    if district_type == 'cd':
        pop_dev = 1
        base_defect = 47
    elif district_type == 'sldu':
        pop_dev = 10
        base_defect = 26
    elif district_type == 'sldl':
        pop_dev = 10
        base_defect = 19
    loose_mask = (df['pop_deviation_plan'] <= pop_dev) & (df['defect_plan'] <= base_defect*3)
    tight_mask = (df['pop_deviation_plan'] <= pop_dev) & (df['defect_plan'] <= base_defect)
    print(f'\n\n{district_type} loose mask stats')
    display(df.loc[loose_mask, 'defect_plan'].describe())
    print(f'{district_type} tight mask stats')
    display(df.loc[tight_mask, 'defect_plan'].describe())
    return df

df_cd   = get_stats('cd')
df_sldl = get_stats('sldu')
df_sldu = get_stats('sldl')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jupyter/MathVGerrmandering_CMAT_2021
Stacking summaries & uniquifying ... 

cd loose mask stats


count    128703.000000
mean         29.889288
std           6.179758
min          19.000000
25%          23.000000
50%          32.000000
75%          35.000000
max          46.000000
Name: defect_plan, dtype: float64

cd tight mask stats


count    23037.000000
mean        21.898164
std          0.333303
min         19.000000
25%         22.000000
50%         22.000000
75%         22.000000
max         22.000000
Name: defect_plan, dtype: float64

Stacking summaries & uniquifying ... 

sldu loose mask stats


...

count    27540.000000
mean        10.440668
std          1.682527
min          7.000000
25%          9.000000
50%         11.000000
75%         12.000000
max         13.000000
Name: defect_plan, dtype: float64

Stacking summaries & uniquifying ... 

sldl loose mask stats


count    406310.000000
mean         37.746408
std          11.436129
min          11.000000
25%          33.000000
50%          40.000000
75%          46.000000
max          57.000000
Name: defect_plan, dtype: float64

sldl tight mask stats


count    73298.000000
mean        17.001187
std          1.799305
min         11.000000
25%         16.000000
50%         17.000000
75%         18.000000
max         19.000000
Name: defect_plan, dtype: float64

In [54]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
from src.nodes import *
from src.mcmc import *

nodes_opts = {
    'district_type'    : 'sldu',
    'contract_thresh'  : 10,
    'level'            : 'cntyvtd',
    'abbr'             : 'TX',
    'proposal'         : 'PLANS2101'
}

N = Nodes(**nodes_opts)
M = MCMC(nodes_tbl=N.tbl, random_seed=9000000)
M.run_chain()

# N.tbl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jupyter/MathVGerrmandering_CMAT_2021
Get crosswalks_TX_2010_tabblock   ... tabblock table exists ... success
Get assignments_TX_2020_tabblock  ... tabblock table exists ... success
Get shapes_TX_2020_tabblock       ... tabblock table exists ... success
Get census_TX_2020_tabblock       ... tabblock table exists ... success
Get elections_TX_2020_tabblock    ... tabblock table exists ... success
Get nodes_TX_2020_cntyvtd_sldu_contract10_plans2101 ... dataframe exists ... cntyvtd table exists ... success


In [4]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *

tbl = 'cmat-315920.redistricting_data.nodes_TX_2020_tabblock_sldu_contract0_plans2101'
query = f"select distinct sldu, vtd from {tbl}"
df = run_query(query)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jupyter/MathVGerrmandering_CMAT_2021


In [16]:
M

NameError: name 'M' is not defined

In [38]:
tbl = 'cmat-315920.redistricting_data.assignments_TX_2020_tabblock'
query = f"select distinct cast(geoid as int) as geoid, sldu, cd, sldl, cntyvtd from {tbl}"
df1 = run_query(query)

# file = data_path / f'assignments/TX/sldu/PLANS2101.csv'
# df2 = pd.read_csv(file)
# df2.columns = ['geoid', 'sldu']

In [35]:
df1

,geoid,cntyvtd
0,484230014041000,423000005
1,484230014041001,423000005
2,484230014041002,423000005
3,484230014041003,423000005
4,484230014041004,423000019
...,...,...
668752,482013240003017,201000347
668753,482013240003018,201000347
668754,482013240003019,201000347
668755,482013240003020,201000347


In [25]:
df2.dtypes

geoid    int64
sldu     int64
dtype: object

In [33]:
df = df1.merge(df2, on='geoid')

In [40]:
w = df1.groupby('cntyvtd')['cd'].nunique()
w[w>1]

cntyvtd
029001102    2
029002039    2
029003086    2
029003125    2
029003199    2
029003206    2
029004007    2
029004044    2
085000013    2
091000202    2
091000406    2
113001071    2
113001087    2
113002807    2
113002810    2
113003940    2
113004608    2
157002138    2
157002139    2
201000045    2
209000227    2
209000228    2
209000230    2
209000232    2
209000333    2
209000335    2
209000339    2
209000417    2
209000420    2
209000449    2
439001277    2
439001441    2
439002461    2
439004067    2
439004245    2
497000008    2
Name: cd, dtype: int64

Andrea - Call me

Andrea, This is a collaboration, but I feel like you have been attempt to issue increasingly direct orders to me.  We can talk out our differences, but the conversation must go both ways.  I have been working on this thing for 3 years.  My students wrote much of GerryChain.  I am an expert here.  We can talk, but I'm not okay with you attempting exert unilateral control.

The reason I have poured years of my life into this is that I believe there are features of the current map that were done to gain unfair political advantage.  As a result, I'm not willing to blindly accept the decisions that were made and expressed in the 2010 map.  I work in a county that was split even though it is far below 1 seats share.  Almost everyone agrees this was a gerrymander.  Your proposal perpetuates that.

Moreover, I ran the defect analysis you asked for on the prior congressional and senate maps.  Nearly everything in the ensemble shows defect 2-3 times what the currently enacted map has.  I'm not okay with that, especially since we can easily do better.

In [51]:
f = N.pq
# f.with_suffix('') / 'hi.pq'
f.parent / f'{f.stem}_step10'

PosixPath('/home/jupyter/redistricting_data/nodes/TX/nodes_TX_2020_cntyvtd_sldu_contract10_step10')

In [134]:
district_type = 'sldu'
n = 31

target_pop = 29145505 / n
seats_col = f'seats_{district_type}'
results = f'cmat-315920.TX_2020_cntyvtd_{district_type}.TX_2020_cntyvtd_{district_type}_0000000_allresults'
nodes = f'cmat-315920.redistricting_data.nodes_TX_2020_cntyvtd_{district_type}_contract2010'
defects = f'cmat-315920.redistricting_data.contract_compares_{district_type}'

query = f"""
select
    seed,
    plan,
    hash_plan,
    max(pop_deviation) as pop_deviation_plan,
    sum(whole_defect) as whole_defect_plan,
    sum(intersect_defect) as intersect_defect_plan,
    sum(defect) as defect_plan
from (
    select
        *,
        whole_defect + intersect_defect as defect
    from (
        select
            *,
            abs(cast(floor({seats_col}) - wholes as int)) as whole_defect,
            abs(cast(intersects - ceiling({seats_col}) as int)) as intersect_defect
        from (
            select
                seed,
                plan,
                hash_plan,
                cnty,
                max(pop_deviation) as pop_deviation,
                sum({seats_col}) as {seats_col},
                sum(whole) as wholes,
                count(*) as intersects
            from (
                select
                    *,
                    case when count(distinct cnty) over (partition by seed, plan, {district_type}) = 1 then 1 else 0 end as whole
                from (
                    select
                        A.seed,
                        A.plan,
                        A.hash_plan,
                        A.cnty,
                        A.{district_type},
                        max(A.pop_deviation) as pop_deviation,
                        sum(B.{seats_col}) as {seats_col}
                    from (
                        select
                            seed,
                            plan,
                            hash_plan,
                            substring(geoid, 1, 3) as cnty,
                            {district_type},
                            geoid,
                            pop_imbalance_plan as pop_deviation
                        from
                            {results}_stack
                        ) as A
                    inner join
                        {nodes} as B
                    on
                        A.geoid = B.geoid
                    group by
                        seed, plan, hash_plan, cnty, {district_type}
                    )
                )
            group by
                seed, plan, hash_plan, cnty
            )
        )
    )
group by
    seed, plan, hash_plan
"""
df = load_table(defects, query=query)

In [135]:
query = f"""
select
    A.seed as random_seed,
    A.plan,
    A.{district_type},
    A.hash_plan,
    B.whole_defect_plan,
    B.intersect_defect_plan,
    B.defect_plan,
    B.pop_deviation_plan,
    A.* except (seed, plan, {district_type}, hash_plan, pop_imbalance_plan, nodes_plan, nodes_district)
from
    {results}_backup as A
inner join
    {output} as B
on
    A.seed = B.seed and A.plan = B.plan
"""
load_table(tbl=results, query=query)

'cmat-315920.TX_2020_cntyvtd_sldu.TX_2020_cntyvtd_sldu_0000000_allresults'

In [ ]:

pop = 29145505
n = 38
target_pop = pop / n
district_type = 'cd'
seats_col = f'seats_{district_type}'
results_old = f'cmat-315920.TX_2020_cntyvtd_cd.TX_2020_cntyvtd_{district_type}_0000000_allresults_stack'
nodes = f'cmat-315920.redistricting_data.nodes_TX_2020_cntyvtd_{district_type}_contract2010'
output = f'cmat-315920.redistricting_data.contract_compares'

query = f"""
select
    seed,
    plan,
    max(pop_deviation) as pop_deviation,
    sum(wholes_defect) as wholes_defect,
    sum(intersects_defect) as intersects_defect,
    sum(defect) as defect
from (
    select
        *,
        wholes_defect + intersects_defect as defect
    from (
        select
            *,
            abs(cast(floor({seats_col}) - wholes as int)) as wholes_defect,
            abs(cast(intersects - ceiling({seats_col}) as int)) as intersects_defect
        from (
            select
                seed,
                plan,
                cnty,
                max(pop_deviation) as pop_deviation,
                sum({seats_col}) as {seats_col},
                sum(whole) as wholes,
                count(*) as intersects
            from (
                select
                    *,
                    case when count(distinct cnty) over (partition by seed, plan, cd) = 1 then 1 else 0 end as whole
                from (
                    select
                        A.seed,
                        A.plan,
                        A.cnty,
                        A.cd,
                        max(A.pop_deviation) as pop_deviation,
                        sum(B.{seats_col}) as {seats_col}
                    from (
                        select
                            seed,
                            plan,
                            substring(geoid, 1, 3) as cnty,
                            cd,
                            geoid,
                            pop_imbalance_plan as pop_deviation
                        from
                            {results_old}
                        --where seed = 1126 and plan = 552
                        ) as A
                    inner join
                        {nodes} as B
                    on
                        A.geoid = B.geoid
                    group by
                        seed, plan, cnty, cd
                    )
                )
            group by
                seed, plan, cnty
            )
        )
    )
group by
    seed, plan
"""
df = load_table(output, query=query)

In [111]:
query = f"""
select
    *
from
    {output}
"""
df = run_query(query)

In [112]:
df['defect'].describe()

count    1.204997e+06
mean     5.795338e+01
std      4.212773e+00
min      3.700000e+01
25%      5.500000e+01
50%      5.800000e+01
75%      6.100000e+01
max      7.500000e+01
Name: defect, dtype: float64

In [ ]:
mask = df['']
df['']

In [92]:
df.sort_values('whole')

,seed,plan,cnty,pop_deviation,seats_cd,whole,intersects
0,1126,552,085,7.060746,1.387853,0,5
160,1126,552,443,7.060746,0.000991,0,1
161,1126,552,035,7.060746,0.023775,0,1
162,1126,552,031,7.060746,0.014829,0,1
163,1126,552,221,7.060746,0.080312,0,1
...,...,...,...,...,...,...,...
125,1126,552,141,7.060746,1.128646,1,2
15,1126,552,453,7.060746,1.682151,1,5
187,1126,552,439,7.060746,2.751859,1,5
41,1126,552,113,7.060746,3.407540,2,7


In [86]:
df[seats_col].sum()

38.00000000000003

In [55]:
pop = 29145505
n = 38
target_pop = pop / n
results_old = 'cmat-315920.TX_2020_cntyvtd_cd.TX_2020_cntyvtd_cd_0000000_allresults_stack'
assignments = 'cmat-315920.redistricting_data.assignments_TX_2020_tabblock'

query = f"""
select
    seed,
    plan,
    cnty,
    max(pop_deviation) as pop_deviation,
    sum(whole) as whole
from (    
    select
        *,
        case when count(distinct cnty) over (partition by seed, plan, cd) = 1 then 1 else 0 end as whole
    from (
        select
            seed,
            plan,
            cnty,
            cd,
            max(pop_deviation) as pop_deviation
        from (
            select
                seed,
                plan,
                substring(geoid, 1, 3) as cnty,
                cd,
                geoid,
                pop_imbalance_plan as pop_deviation
            from
                {results_old}
            where
                seed = 1126 and plan = 552
            )
        group by
            seed, plan, cnty, cd
        )
    )
group by
    seed, plan, cnty
"""

df = run_query(query)
#     count(*) over (partition byhash_plan,
df

,seed,plan,cnty,pop_deviation,whole
0,1126,552,121,7.060746,0
1,1126,552,183,7.060746,0
2,1126,552,423,7.060746,0
3,1126,552,321,7.060746,0
4,1126,552,157,7.060746,0
...,...,...,...,...,...
249,1126,552,103,7.060746,0
250,1126,552,019,7.060746,0
251,1126,552,457,7.060746,0
252,1126,552,277,7.060746,0


In [26]:
f = f'nodes_TX_2020_tabblock_sldu_contract0'
nodes_tbl = f'{proj_id}.redistricting_data.{f}'
pq = root_path / f'redistricting_data/nodes/TX/{f}.parquet'
pq.parent.mkdir(parents=True, exist_ok=True)
q = f"select * from {nodes_tbl}"
df = run_query(q)
df.to_parquet(pq)
to_gcs(pq)

In [27]:
f = f'nodes_TX_2020_tabblock_sldl_contract0'
nodes_tbl = f'{proj_id}.redistricting_data.{f}'
pq = root_path / f'redistricting_data/nodes/TX/{f}.parquet'
pq.parent.mkdir(parents=True, exist_ok=True)
q = f"select * from {nodes_tbl}"
df = run_query(q)
df.to_parquet(pq)
to_gcs(pq)